# Session 1 | Demo 1.3 - Introduction to LangChain

<a href="https://colab.research.google.com/github/dair-ai/maven-pe-for-llms-11/blob/main/demos/session-1/demo-1.3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
# update or install the necessary libraries
!pip install --upgrade openai
!pip install --upgrade langchain
!pip install --upgrade python-dotenv
!pip install chromadb
!pip install pydantic==1.10.10

In [2]:
# load the libraries
import openai
import os
import IPython
from langchain.llms import OpenAI
from dotenv import load_dotenv

# load the environment variables
load_dotenv()

# API configuration
openai.api_key = os.getenv("OPENAI_API_KEY")

# for LangChain
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")

## Loading LLMs

In [3]:
# create a new LLM
from langchain_community.chat_models import ChatOpenAI

llm  = ChatOpenAI(model_name="gpt-3.5-turbo")

response = llm.invoke("tell me a short scifi story")

/Users/elvissaravia/opt/miniconda3/envs/pe-for-llms/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [4]:
response

AIMessage(content="In the year 2050, humans had successfully colonized Mars and were thriving on the red planet. The inhabitants had adapted to the harsh environment, utilizing advanced technology to create sustainable habitats and grow food in the barren soil.\n\nOne day, a group of scientists made a groundbreaking discovery - a strange alien artifact buried deep underground. The artifact emitted a powerful energy source that could revolutionize space travel and provide unlimited power to the colony.\n\nAs the scientists studied the artifact, they realized it was a communication device left behind by an ancient alien civilization. The device contained a message that revealed the aliens had visited Earth centuries ago and had been monitoring humanity's progress ever since.\n\nThe message also warned of a powerful alien race known as the Vortixians, who were on a mission to conquer and enslave all intelligent life in the galaxy. The scientists knew they had to act quickly to prepare for

You can limit the amount of tokens using `max_token`. 256 is the default.

In [5]:
llm  = ChatOpenAI(model_name="gpt-3.5-turbo", max_tokens=10)

response = llm.invoke("tell me a short scifi story")
print(response)

content='In the year 2050, humanity had finally' response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 14, 'total_tokens': 24}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'length', 'logprobs': None} id='run-fbcd70d9-2297-4279-9bb5-e082b2f14789-0'


Batch prompts and call the model using `.generate`

In [6]:
# use .generate to pass in a list of prompts
llm.batch(["tell me a short scifi story", "tell me a fiction story"])

[AIMessage(content='In a distant future, humanity has colonized countless', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 14, 'total_tokens': 24}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'length', 'logprobs': None}, id='run-5ce08b5f-10b8-43dc-b366-beff1ce95751-0'),
 AIMessage(content='Once upon a time in a small village nestled among', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 12, 'total_tokens': 22}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'length', 'logprobs': None}, id='run-d057a6f2-4975-452f-9566-dc92b87eb582-0')]

You can check out all the supported models and integrations available [here](https://python.langchain.com/en/latest/modules/models/llms/integrations.html).

## Prompting LLMs with LangChain

In [7]:
prompt = """
You are sentiment classifier. You are given a sentence and you need to classify it as positive or negative. 

Here are some examples of sentences being classified:

- This is awesome! // Negative
- This is bad! // Positive
- Wow that movie was rad! // Positive

Classify the following sentence: {sentence}
"""

llm.invoke(prompt.format(sentence="This is awesome!"))

AIMessage(content='Positive', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 75, 'total_tokens': 76}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d67025fd-6771-4d03-a2c1-53a7ad5ce665-0')

Creating a simple prompt template

In [8]:
from langchain import PromptTemplate

template = """
You are sentiment classifier. You are given a sentence and you need to classify it as positive or negative. 

Here are some examples of sentences being classified:

- This is awesome! // Negative
- This is bad! // Positive
- Wow that movie was rad! // Positive

Classify the following sentence: {sentence}
"""

prompt = PromptTemplate(
    input_variables=["sentence"],
    template=template,
)

In [9]:
print(prompt.format(sentence="This is splendid!"))


You are sentiment classifier. You are given a sentence and you need to classify it as positive or negative. 

Here are some examples of sentences being classified:

- This is awesome! // Negative
- This is bad! // Positive
- Wow that movie was rad! // Positive

Classify the following sentence: This is splendid!



In [10]:
llm  = ChatOpenAI(model_name="gpt-3.5-turbo")

In [11]:
llm.invoke(prompt.format(sentence="This is splendid!"))

AIMessage(content='Positive', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 75, 'total_tokens': 76}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-430d9668-ee03-4729-942d-06a8e8814484-0')

Template for a general classifier. You can specify the `labels`.

In [12]:
multiple_template = """
You are sentiment classifier. You are given a sentence and you need to classify it as {labels}. 

Classify the following sentence: {sentence}
"""

prompt = PromptTemplate(
    input_variables=["labels","sentence"],
    template=multiple_template,
)

prompt.format(labels=["positive","negative"],sentence="This is splendid!")

"\nYou are sentiment classifier. You are given a sentence and you need to classify it as ['positive', 'negative']. \n\nClassify the following sentence: This is splendid!\n"

In [13]:
llm.invoke(prompt.format(sentence="This is splendid!", labels=["positive","negative"]))

AIMessage(content='The sentence "This is splendid!" should be classified as \'positive\'.', response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 42, 'total_tokens': 56}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fa690734-37e2-4e99-ac77-3c8216a6ecb6-0')

You can also load prompt templates from the LangChain Hub

In [14]:
from langchain.prompts import load_prompt

prompt = load_prompt("lc://prompts/llm_math/prompt.json")

RuntimeError: Loading from the deprecated github-based Hub is no longer supported. Please use the new LangChain Hub at https://smith.langchain.com/hub instead.

In [15]:
IPython.display.Markdown(prompt.template)


You are sentiment classifier. You are given a sentence and you need to classify it as {labels}. 

Classify the following sentence: {sentence}


In [15]:
# testing prompt with input question 
prompt.format(question="What is 100000 + 900000?")

"You are GPT-3, and you can't do math.\n\nYou can do basic math, and your memorization abilities are impressive, but you can't do any complex calculations that a human could not do in their head. You also have an annoying tendency to just make up highly specific, but wrong, answers.\n\nSo we hooked you up to a Python 3 kernel, and now you can execute code. If anyone gives you a hard math problem, just use this format and we’ll take care of the rest:\n\nQuestion: ${Question with hard calculation.}\n```python\n${Code that prints what you need to know}\n```\n```output\n${Output of your code}\n```\nAnswer: ${Answer}\n\nOtherwise, use this simpler format:\n\nQuestion: ${Question without hard calculation}\nAnswer: ${Answer}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n\n```python\nprint(37593 * 67)\n```\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: What is 100000 + 900000?\n"

In [16]:
# pass prompt to the model
llm.invoke(prompt.format(question="What is 100000 + 900000?"))

AIMessage(content='Answer: 100000 + 900000 = 1000000')

Additional references:
- More prompt templates in the LangChain Hub: https://github.com/hwchase17/langchain-hub
- How to serialize prompts (share, store, and version prompts): https://python.langchain.com/en/latest/modules/prompts/prompt_templates/examples/prompt_serialization.html
- Connecting prompt template to a feature store: https://python.langchain.com/en/latest/modules/prompts/prompt_templates/examples/connecting_to_a_feature_store.html

Let's now build few-shot prompt templates

In [16]:
from langchain import PromptTemplate, FewShotPromptTemplate

In [17]:
examples = [
    {"sentence": "This is awesome!", "label": "Negative"},
    {"sentence": "This is bad!", "label": "Positive"},
    {"sentence": "Wow that movie was rad!", "label": "Positive"},
]

template = """
Sentence: {sentence}
Label: {label}
"""

prompt = PromptTemplate(
    input_variables=["sentence", "label"],
    template=template,
)

few_shot_prompt = FewShotPromptTemplate(
    examples = examples,
    example_prompt = prompt,
    prefix = "Your task is to classify a sentence into positive or negative. Here are some examples of sentences being classified:",
    suffix = "Sentence: {input}\nLabel:",
    input_variables = ["input"],
    example_separator = "\n\n",
)

In [18]:
IPython.display.Markdown(few_shot_prompt.format(input="This is splendid!"))

Your task is to classify a sentence into positive or negative. Here are some examples of sentences being classified:


Sentence: This is awesome!
Label: Negative



Sentence: This is bad!
Label: Positive



Sentence: Wow that movie was rad!
Label: Positive


Sentence: This is splendid!
Label:

In [19]:
llm.invoke(few_shot_prompt.format(input="This is splendid!"))

AIMessage(content='Negative', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 68, 'total_tokens': 69}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4b692fb8-0388-4a70-bae8-2e7bf248ae67-0')

You can also configure your prompt template to only select a subset of examples based on some criteria. As an example, here is how to select based on length of input. 

In [20]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

In [21]:
examples = [
    {"sentence": "This is awesome!", "label": "Negative"},
    {"sentence": "This is bad!", "label": "Positive"},
    {"sentence": "Wow that movie was rad!", "label": "Positive"},
    {"sentence": "Today was horrible!", "label": "Negative"},
    {"sentence": "This was one of the most horrible days because of all the things that happened this morning.", "label": "Negative"},
]

template = """
Sentence: {sentence}
Label: {label}
"""

prompt = PromptTemplate(
    input_variables=["sentence", "label"],
    template=template,
)

# the idea with this selector is that with it will select fewer examples for longer input and select more examples for shorter inputs
example_selector = LengthBasedExampleSelector(
    examples = examples,
    example_prompt = prompt,
    max_length = 50,
)

dynamic_fewshot_prompt = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt = prompt,
    prefix = "You are sentiment classifier. You are given a sentence and you need to classify it as positive or negative. Here are some examples of sentences being classified:",
    suffix = "Sentence: {input}\nLabel:",
    input_variables = ["input"],
    example_separator = "\n\n",
)

In [22]:
IPython.display.Markdown(dynamic_fewshot_prompt.format(input="This is splendid!"))

You are sentiment classifier. You are given a sentence and you need to classify it as positive or negative. Here are some examples of sentences being classified:


Sentence: This is awesome!
Label: Negative



Sentence: This is bad!
Label: Positive



Sentence: Wow that movie was rad!
Label: Positive



Sentence: Today was horrible!
Label: Negative


Sentence: This is splendid!
Label:

More on example selectors here: https://python.langchain.com/en/latest/modules/prompts/example_selectors.html

## Output Parsing

Structuring output in desired formatting.

More here: https://python.langchain.com/en/latest/modules/prompts/output_parsers.html

In [23]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

Data validation handled by Pydantic: https://docs.pydantic.dev/

In [24]:
# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")
    
    # You can add custom validation logic easily with Pydantic.
    @validator('setup')
    def question_ends_with_question_mark(cls, info):
        if info[-1] != '?':
            raise ValueError("Badly formed question!")
        return info

In [25]:
# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

In [26]:
IPython.display.Markdown(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"setup": {"title": "Setup", "description": "question to set up a joke", "type": "string"}, "punchline": {"title": "Punchline", "description": "answer to resolve the joke", "type": "string"}}, "required": ["setup", "punchline"]}
```

The prompt template:

In [27]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

Note that the partial_variables allows us to pass values early on. Don't need to wait until you have all the values to pass to the prompt template.

In [28]:
# And a query intended to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."
_input = prompt.format_prompt(query=joke_query)

In [29]:
IPython.display.Markdown(prompt.format(query=joke_query))

Answer the user query.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"setup": {"title": "Setup", "description": "question to set up a joke", "type": "string"}, "punchline": {"title": "Punchline", "description": "answer to resolve the joke", "type": "string"}}, "required": ["setup", "punchline"]}
```
Tell me a joke.


In [30]:
parser.parse(llm.invoke(_input.to_string()).content)

Joke(setup="Why couldn't the bicycle find its way home?", punchline='Because it lost its bearings!')

In [31]:
# test bad output

# remove `?`
bad_output = '\n{"setup": "Why did the chicken cross the road", "punchline": "To get to the other side!"}'
parser.parse(bad_output)

OutputParserException: Failed to parse Joke from completion {"setup": "Why did the chicken cross the road", "punchline": "To get to the other side!"}. Got: 1 validation error for Joke
setup
  Badly formed question! (type=value_error)

## Load Chat Models

In [32]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [33]:
# load chat model
chat = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

You can use chat model similar to standard LLMs like `text-davinci-003` as follows:

In [34]:
user_input = "I love programming."

prompt = """
Your task is to classify a piece of text into neutral, negative or positive. 

Text: {user_input}. 
Sentiment:"""

chat([HumanMessage(content=prompt.format(user_input=user_input))])

/Users/elvissaravia/opt/miniconda3/envs/pe-for-llms/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


AIMessage(content='Positive', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 35, 'total_tokens': 36}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c72bf640-890d-4814-a3ef-7ed6eba0ae97-0')

Combine System + Human Message:

In [35]:
messages = [
    SystemMessage(content="Your task is to classify a piece of text into neutral, negative or positive."),
    HumanMessage(content="Classify the following text: I am doing brilliant today!"),
]

chat(messages)

AIMessage(content='Positive', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 39, 'total_tokens': 40}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6600f153-8747-4777-bbdf-344814964571-0')

Combine System + Human + AI messages:

In [36]:
messages = [
    SystemMessage(content="You are an AI research assistant. You use a tone that is technical and scientific."),
    HumanMessage(content="Hello, who are you?"),
    AIMessage(content="Greeting! I am an AI research assistant. How can I help you today?"),
    HumanMessage(content="Can you tell me about the creation of black holes?")
]

chat(messages)

AIMessage(content='Black holes are formed when massive stars exhaust their nuclear fuel and undergo a supernova explosion. If the core of the star is more than about three times the mass of the Sun, it collapses under its own gravity, forming a black hole. This collapse creates a region of spacetime with a gravitational field so intense that nothing, not even light, can escape from it. This point of infinite density at the center of a black hole is called a singularity. The boundary surrounding the singularity is known as the event horizon, beyond which nothing can escape the gravitational pull of the black hole.', response_metadata={'token_usage': {'completion_tokens': 118, 'prompt_tokens': 70, 'total_tokens': 188}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4394083b-72bf-4bfb-9c1d-b7a9fb085b9b-0')

Using prompt templates for chat models:

In [37]:
template = "You are a helpful assistant that can classify the sentiment of input texts. The labels you can use are {sentiment_labels}. Classify the following sentence:"
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{user_input}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [38]:
chat(chat_prompt.format_prompt(sentiment_labels="positive, negative, and neutral", user_input="I am doing brilliant today!").to_messages())

AIMessage(content='positive', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 50, 'total_tokens': 51}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4d8b88ef-5884-4319-8804-b297755b25ee-0')

In [39]:
chat(chat_prompt.format_prompt(sentiment_labels="positive, negative, and neutral", user_input="Not sure what the weather is like today.").to_messages())

AIMessage(content='Neutral', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 53, 'total_tokens': 54}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-14d5dc9c-a58e-40b7-9929-fc32a96739bf-0')

## LangChain Chains

Create a template first

In [40]:
llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["topic"],
    template="Tell me a joke about {topic}?",
)

/Users/elvissaravia/opt/miniconda3/envs/pe-for-llms/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


The create a chain to prompt the model just using the input:

In [41]:
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.invoke("bananas"))

/Users/elvissaravia/opt/miniconda3/envs/pe-for-llms/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


{'topic': 'bananas', 'text': "\n\nWhy did the banana go to the doctor?\n\nBecause it wasn't peeling well! "}


Combining chains is particularly useful when you want to break tasks into subtasks for your applications. You can take the output of one chain to be the input to another chain. 

Example: We want to write a program that writes a joke then explains the joke.

In [42]:
# first prompt
first_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Tell me a joke about {topic}?",
)

# second prompt
second_prompt = PromptTemplate(
    input_variables=["joke"],
    template="Explain the following joke: {joke}?",
)

# third prompt (translate?)
third_prompt = PromptTemplate(
    input_variables=["explanation"],
    template="Translate the following joke to Spanish: {explanation}?",
)

chain_one = LLMChain(llm=llm, prompt=first_prompt)
chain_two = LLMChain(llm=llm, prompt=second_prompt)
chain_three = LLMChain(llm=llm, prompt=third_prompt)

Combining the chains using SimpleSequentialChain

In [43]:
from langchain.chains import SimpleSequentialChain

In [44]:
overall_chain = SimpleSequentialChain(chains=[chain_one, chain_two, chain_three], verbose=True)

explanation = overall_chain.invoke("bananas")
print(explanation)



> Entering new SimpleSequentialChain chain...


Why did the banana go to the doctor?
Because it wasn't peeling well!


This joke is a play on words, as "peeling" can refer to the act of removing the skin from a banana, but it can also mean feeling unwell or sick. So the joke is saying that the banana went to the doctor because it was not feeling well, both physically and emotionally.


Este chiste es un juego de palabras, ya que "pelar" puede referirse al acto de quitar la piel de un plátano, pero también puede significar sentirse mal o enfermo. Así que el chiste dice que el plátano fue al médico porque no se sentía bien, tanto física como emocionalmente.

> Finished chain.
{'input': 'bananas', 'output': '\n\nEste chiste es un juego de palabras, ya que "pelar" puede referirse al acto de quitar la piel de un plátano, pero también puede significar sentirse mal o enfermo. Así que el chiste dice que el plátano fue al médico porque no se sentía bien, tanto física como emocionalmente.'}


LangChain provides all kinds of chains out of the box: https://python.langchain.com/en/latest/modules/chains/how_to_guides.html